In [508]:
import pyodbc
import pandas as pd
import numpy as np
from numpy import median

#logging in to xpressfeed using desktop credentials
QA_SERVER = 'qaidbsql01\qa2008'
QA_DB = 'Xpressfeed'
_SQL_SERVER_DRIVER = '{SQL Server}'

server = QA_SERVER
database = QA_DB
connStr = 'DRIVER=' + _SQL_SERVER_DRIVER + ';' + 'SERVER={0};DATABASE={1};TDS_Version=8.0;Trusted_Connection=yes;'.format(server, database)
conn = pyodbc.connect(connStr)

In [392]:
df_ticker = pd.read_excel('Ticker List.xlsx', sheet_name = "Tickers")
df_ticker


,CIQ ID,Ticker,Company Name
0,21835,NasdaqGS:MSFT,Microsoft Corporation
1,24937,NasdaqGS:AAPL,Apple Inc.
2,18749,NasdaqGS:AMZN,"Amazon.com, Inc."
3,29096,NasdaqGS:GOOG.L,Alphabet Inc.
4,668212,CCSE:MVZ.B,"Mercantil Servicios Financieros, C.A."
5,255251,NYSE:BRK.A,Berkshire Hathaway Inc.
6,20765463,NasdaqGS:FB,"Facebook, Inc."
7,42083601,NYSE:BABA,Alibaba Group Holding Limited
8,11042136,SEHK:700,Tencent Holdings Limited
9,139677,NYSE:JNJ,Johnson & Johnson


In [393]:
a = df_ticker["CIQ ID"].apply(str).tolist()
b = a[:10]
b

['21835',
 '24937',
 '18749',
 '29096',
 '668212',
 '255251',
 '20765463',
 '42083601',
 '11042136',
 '139677']

In [66]:
#One Company and one line item

#company id
ciq_id = "24937"

#line item
line_item = '1169'
line_item_name = "Gross PP&E"

#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = '2018'

#latest filing
financial_flag = '1'


query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, fp.fiscalchainSeriesId as fiscalChainSeriesId,' \
                    'fp.fiscalQuarter, fp.fiscalYear, fd.dataItemValue * fu.unitTypeValue as data_item ' \
                    'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                    'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                    'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                    'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                    'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                    'WHERE c.companyId={0} AND fd.dataItemId={1} AND fp.periodTypeId={2} AND ' \
                    'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(ciq_id, line_item, period_type, fiscal_year, financial_flag)
                    
df = pd.read_sql(query_str, conn)
df

,companyId,periodEndDate,filingDate,fiscalChainSeriesId,fiscalQuarter,fiscalYear,data_item
0,24937,2018-09-29,2018-11-05,1,4,2018,9.040300e+10


In [225]:
#Mutiple Companies and Multiple Line Items

#company IDs
ciq_id = b

#list of line items
line_item = ["112", "2150", "2160", "82", "3168", "3170", "1043", "3140", "83", "47", "19", "1001", "1212", "1009",
                "1046", "1279", "3160", "3148", "21678", "1169", "1002", "3098", "1272", "1267", "2171"]

#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = '2018'

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)

                   
df = pd.read_sql(query_str, conn)
df

,companyId,periodEndDate,filingDate,fiscalQuarter,fiscalYear,dataItemId,data_item,dataItemName
0,24937,2018-09-29,2018-11-05,4,2018,19,2.000000e+07,"Unusual Items, Total"
1,18749,2018-12-31,2019-02-01,4,2018,19,6.000000e+06,"Unusual Items, Total"
2,21835,2018-06-30,2018-08-03,4,2018,19,2.392000e+09,"Unusual Items, Total"
3,29096,2018-12-31,2019-02-05,4,2018,19,1.459000e+09,"Unusual Items, Total"
4,139677,2018-12-30,2019-02-20,4,2018,19,-2.643000e+09,"Unusual Items, Total"
5,42083601,2018-03-31,2018-07-27,4,2018,19,-1.295000e+09,"Unusual Items, Total"
6,11042136,2018-12-31,2019-03-21,4,2018,19,1.257400e+10,"Unusual Items, Total"
7,18749,2018-12-31,2019-02-01,4,2018,47,9.000000e+06,Income (Loss) On Equity Invest.
8,255251,2018-12-31,2019-02-25,4,2018,47,-2.167000e+09,Income (Loss) On Equity Invest.
9,42083601,2018-03-31,2018-07-27,4,2018,47,-2.079200e+10,Income (Loss) On Equity Invest.


In [235]:
len(b)

10

In [237]:
len(line_item)

25

In [234]:
df.sort_values(["companyId", "data_item"])

,companyId,periodEndDate,filingDate,fiscalQuarter,fiscalYear,dataItemId,data_item,dataItemName
12,18749,2018-12-31,2019-02-01,4,2018,82,-1.417000e+09,"Interest Expense, Total"
1,18749,2018-12-31,2019-02-01,4,2018,19,6.000000e+06,"Unusual Items, Total"
7,18749,2018-12-31,2019-02-01,4,2018,47,9.000000e+06,Income (Loss) On Equity Invest.
90,18749,2018-12-31,2019-02-01,4,2018,1267,4.180000e+08,Other Current Assets (Summary Subtotal)
166,18749,2018-12-31,2019-02-01,4,2018,3170,3.400000e+09,"Net Rental Expense, Total"
99,18749,2018-12-31,2019-02-01,4,2018,1272,6.652000e+09,"Other Assets, Total"
109,18749,2018-12-31,2019-02-01,4,2018,1279,9.502000e+09,Current Portion of Long-Term Debt / Capital Le...
115,18749,2018-12-31,2019-02-01,4,2018,2150,1.007300e+10,Net Income - (CF)
134,18749,2018-12-31,2019-02-01,4,2018,2171,1.486600e+10,Depreciation & Amortization - CF
125,18749,2018-12-31,2019-02-01,4,2018,2160,1.534100e+10,"Depreciation & Amortization, Total - CF"


In [80]:
len(line_item)

34

In [27]:
#Mutiple Companies and Multiple Line Items

#company IDs
ciq_id = ["24937"]

#list of line items
line_item = ["112", "2150", "2160", "82", "3168", "3170", "1002", "1043", "3140", "83", "47", "19", "1001", "1212", "1009"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = '2018'

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)

                    
df = pd.read_sql(query_str, conn)
df

,companyId,periodEndDate,filingDate,fiscalQuarter,fiscalYear,dataItemId,data_item,dataItemName
0,24937,2018-09-29,2018-11-05,4,2018,19,2.000000e+07,"Unusual Items, Total"
1,24937,2018-09-29,2018-11-05,4,2018,82,-3.240000e+09,"Interest Expense, Total"
2,24937,2018-09-29,2018-11-05,4,2018,112,2.655950e+11,Revenues
3,24937,2018-09-29,2018-11-05,4,2018,3168,1.423600e+10,Research And Development Expense From Footnotes
4,24937,2018-09-29,2018-11-05,4,2018,3170,1.200000e+09,"Net Rental Expense, Total"
5,24937,2018-09-29,2018-11-05,4,2018,1001,4.899500e+10,Total Receivables
6,24937,2018-09-29,2018-11-05,4,2018,1009,1.168660e+11,Total Current Liabilities
7,24937,2018-09-29,2018-11-05,4,2018,1043,3.956000e+09,Inventory
8,24937,2018-09-29,2018-11-05,4,2018,2150,5.953100e+10,Net Income - (CF)
9,24937,2018-09-29,2018-11-05,4,2018,2160,1.090300e+10,"Depreciation & Amortization, Total - CF"


In [88]:
#Mutiple Companies and Multiple Line Items

#company IDs
ciq_id = ["313055"]

#list of line items
line_item = ["112", "2150", "2160", "82", "3168", "3170", "1002", "1043", "3140", "83", "47", "19", "1001", "1212", "1009",
                "1046", "1279", "3160", "3148", "21678", "1169", "1002", "1001", "1212", "1009", "1046", "1279", "3098","1043",
                  "1272", "1267", "2171", "3170", "3168"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = '2018'

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)

                    
df = pd.read_sql(query_str, conn)
df

,companyId,periodEndDate,filingDate,fiscalQuarter,fiscalYear,dataItemId,data_item,dataItemName
0,313055,2018-01-31,2018-03-30,4,2018,1001,5.614000e+09,Total Receivables
1,313055,2018-01-31,2018-03-30,4,2018,1009,7.852100e+10,Total Current Liabilities
2,313055,2018-01-31,2018-03-30,4,2018,1043,4.378300e+10,Inventory
3,313055,2018-01-31,2018-03-30,4,2018,1046,5.257000e+09,Short-term Borrowings
4,313055,2018-01-31,2018-03-30,4,2018,1169,1.978570e+11,Gross Property Plant And Equipment
5,313055,2018-01-31,2018-03-30,4,2018,1212,3.511000e+09,Prepaid Expenses
6,313055,2018-01-31,2018-03-30,4,2018,1272,9.790000e+09,"Other Assets, Total"
7,313055,2018-01-31,2018-03-30,4,2018,1279,4.405000e+09,Current Portion of Long-Term Debt / Capital Le...
8,313055,2018-01-31,2018-03-30,4,2018,3098,2.529800e+10,Land - (BS)
9,313055,2018-01-31,2018-03-30,4,2018,2150,9.862000e+09,Net Income - (CF)


In [131]:

line_item_list = ["112", "2150", "2160", "82", "3168", "3170", "1002", "1043", "3140", "83", "47", "19", "1001", "1212", "1009",
                "1046", "1279", "3160", "3148", "21678", "1169", "1002", "1001", "1212", "1009", "1046", "1279", "3098","1043",
                  "1272", "1267", "2171", "3170", "3168"]

line_item_list = sorted(line_item_list)
line_item_int = [int(i) for i in line_item_list]
print(line_item_int)
query_str = 'SELECT * FROM Xpressfeed..ciqDataItem WHERE dataItemId IN ({0})'.format(', '.join(line_item_list))

df = pd.read_sql(query_str, conn)
xpress_list = df['dataItemId'].tolist()
xpress_list = [int(i) for i in xpress_list]

print(xpress_list)
print(len(xpress_list))
print(len(line_item_int))
print(set(line_item_int))
print(set(xpress_list))



[1001, 1001, 1002, 1002, 1009, 1009, 1043, 1043, 1046, 1046, 112, 1169, 1212, 1212, 1267, 1272, 1279, 1279, 19, 2150, 2160, 21678, 2171, 3098, 3140, 3148, 3160, 3168, 3168, 3170, 3170, 47, 82, 83]
[19, 47, 82, 83, 112, 1001, 1002, 1009, 1043, 1046, 1169, 1212, 1267, 1272, 1279, 2150, 2160, 2171, 3098, 3140, 3148, 3160, 3168, 3170, 21678]
25
34
{1169, 1043, 19, 1046, 3098, 21678, 47, 1212, 3140, 3148, 82, 83, 3160, 3168, 3170, 2150, 1001, 1002, 112, 1009, 2160, 1267, 1272, 2171, 1279}
{1169, 19, 1043, 1046, 3098, 21678, 47, 1212, 3140, 3148, 82, 83, 3160, 3168, 3170, 2150, 1001, 1002, 112, 1009, 2160, 1267, 1272, 2171, 1279}


In [46]:
#Mutiple Companies and Multiple Line Items

#company IDs
ciq_id = ["313055"]

#list of line items
line_item = ["3160", "3148", "21678", "1212"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = '2018'

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)

                    
df = pd.read_sql(query_str, conn)
df

,companyId,periodEndDate,filingDate,fiscalQuarter,fiscalYear,dataItemId,data_item,dataItemName
0,313055,2018-01-31,2018-03-30,4,2018,1212,3.511000e+09,Prepaid Expenses


In [186]:
#Mutiple Companies and Multiple Line Items

#company IDs
ciq_id = ["24937"]

#list of line items
line_item = ["10"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = '2018'

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)

                    
df = pd.read_sql(query_str, conn)
df

,companyId,periodEndDate,filingDate,fiscalQuarter,fiscalYear,dataItemId,data_item,dataItemName
0,24937,2018-09-29,2018-11-05,4,2018,10,1.018390e+11,Gross Profit


In [40]:
len(line_item)

8

In [137]:
#list of mapping CIQ IDs to SQL database codes

# IQ_REV = 112
# IQ_NI_CF = 2150
# IQ_DA_CF = 2160
# IQ_INTEREST_EXP = 82
# IQ_RD_EXP_FN = 3168
# IQ_NET_RENTAL_EXP_FN = 3170
# IQ_CASH_ST_INVEST = 1002
# IQ_INVENTORY = 1043
# IQ_PENSION_TOTAL_EXP = 3140
# IQ_MINORITY_INTEREST_IS = 83
# IQ_INC_EQUITY = 47
# IQ_TOTAL_UNUSUAL = 19
# IQ_TOTAL_RECEIV = 1001
# IQ_PREPAID_EXP = 1212
# IQ_TOTAL_CL = 1009
# IQ_ST_DEBT = 1046
# IQ_CURRENT_PORT = 1279
# IQ_PENSION_SERVICE_COST = 3160
# IQ_PENSION_INTEREST_COST = 3148
# IQ_NONCASH_PENSION_EXP = 21678
# IQ_GPPE = 1169
# IQ_CASH_ST_INVEST = 1002
# IQ_TOTAL_RECEIV = 1001
# IQ_PREPAID_EXP = 1212
# IQ_TOTAL_CL = 1009
# IQ_ST_DEBT = 1046
# IQ_CURRENT_PORT = 1279
# IQ_LAND = 3098
# IQ_INVENTORY = 1043
# IQ_OTHER_ASSETS = 1272
# IQ_OTHER_CA = 1267
# IQ_DA_SUPPL_CF = 2171
# IQ_NET_RENTAL_EXP_FN = 3170
# IQ_RD_EXP_FN = 3168

line_item_list = ["112", "2150", "2160", "82", "3168", "3170", "1043", "3140", "83", "47", "19", "1001", "1212", "1009",
                "1046", "1279", "3160", "3148", "21678", "1169", "1002", "3098", "1272", "1267", "2171"]

In [138]:
set([x for x in line_item_list if line_item_list.count(x) > 1])

set()

In [410]:
#All data items search

#company IDs
ciq_id = ["24937"]

#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = '2018'

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fp.periodTypeId={1} AND ' \
            'fp.fiscalYear={2} AND fi.latestForFinancialPeriodFlag = {3}'.format(', '.join(ciq_id), period_type, fiscal_year, financial_flag)
df = pd.read_sql(query_str, conn)
df = df.drop(df.columns[[0, 1, 2, 3, 4, 6]], axis=1)  

In [413]:
#Search function to find line item 

keyword = "a"

df[df.apply(lambda row: row.astype(str).str.contains(keyword).any(), axis=1)]

,dataItemId,dataItemName
1,4,"EBT, Excl. Unusual Items"
2,6,Earnings Before Xtra Ord. Item & Accounting Ch...
3,7,Earnings From Continuing Operations
5,9,Net EPS - Basic
8,16,Net Income to Common Incl Extra Items
9,19,"Unusual Items, Total"
10,21,Operating Income
11,23,Selling General & Admin Expenses (Summary Subt...
12,28,Total Revenues
13,34,"Cost of Goods Sold, Total"


In [662]:
year = "2018"
ticker = "280003133"


#Gross Cash Flow calcs

#Gross Cash Flow t0

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_NI_CF = 2150
# IQ_DA_CF = 2160
# IQ_INTEREST_EXP = 82
# IQ_RD_EXP_FN = 3168
# IQ_NET_RENTAL_EXP_FN = 3170
# IQ_PENSION_TOTAL_EXP = 3140
# IQ_PENSION_SERVICE_COST = 3160
# IQ_PENSION_INTEREST_COST = 3148
# IQ_NONCASH_PENSION_EXP = 21678
# IQ_MINORITY_INTEREST_IS = 83
# IQ_INC_EQUITY = 47
# IQ_TOTAL_UNUSUAL = 19

line_item = ["2150", "2160", "82", "3168", "3170", "3140", "3160", "3148", "21678", "83", "47", "19"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = year

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)


df_t0 = pd.read_sql(query_str, conn)


#Gross Cash Flow t-1

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_CASH_ST_INVEST = 1002
# IQ_TOTAL_RECEIV = 1001
# IQ_PREPAID_EXP = 1212
# IQ_TOTAL_CL = 1009
# IQ_ST_DEBT = 1046
# IQ_CURRENT_PORT = 1279
# IQ_INVENTORY = 1043


line_item = ["1002", "1001", "1212", "1009", "1046", "1043", "1279"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = str(int(year) - 1)

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)


df_t1 = pd.read_sql(query_str, conn)

#potential to get better results by using try/except for all values

try:
    net_income = df_t0.loc[df_t0["dataItemId"] == 2150]["data_item"].iloc[0]
except: 
    net_income = 0 

try:
    d_and_a = df_t0.loc[df_t0["dataItemId"] == 2160] ["data_item"].iloc[0]
except:
    d_and_a = 0

try:
    interest_expense =  df_t0.loc[df_t0["dataItemId"] == 82]["data_item"].iloc[0]
except:
    interest_expense = 0 

try:
    r_and_d_expense =  df_t0.loc[df_t0["dataItemId"] == 3168]["data_item"].iloc[0]
except:
    r_and_d_expense = 0

try:
    rental_expense =  df_t0.loc[df_t0["dataItemId"] == 3170]["data_item"].iloc[0]
except:
    rental_expense = 0

try:
    minority_interest = df_t0.loc[df_t0["dataItemId"] == 83]["data_item"].iloc[0]
except:
    minority_interest = 0

try:
    associate_jv_income = df_t0.loc[df_t0["dataItemId"] == 47]["data_item"].iloc[0]
except: 
    associate_jv_income = 0

try:
    special_items = df_t0.loc[df_t0["dataItemId"] == 19]["data_item"].iloc[0]
except:
    special_items = 0

# -gdp t0*(Cash & ST Invt + Receiv + prepaid exp - current liab + short term debt + current portion debt)t-1
try:
    gdp = 0.022
    monetary_holding = -gdp*(df_t1.loc[df_t1["dataItemId"] == 1002]["data_item"].iloc[0] + df_t1.loc[df_t1["dataItemId"] == 1001]["data_item"].iloc[0]
                            + df_t1.loc[df_t1["dataItemId"] == 1212]["data_item"].iloc[0] -  df_t1.loc[df_t1["dataItemId"] == 1009]["data_item"].iloc[0] 
                            + df_t1.loc[df_t1["dataItemId"] == 1046]["data_item"].iloc[0] + df_t1.loc[df_t1["dataItemId"] == 1279]["data_item"].iloc[0])
except:
    monetary_holding = 0

#total pension cost - service cost + interest cost - noncash expense
try:
    pension =  (df_t0.loc[df_t0["dataItemId"] == 3140]["data_item"].iloc[0] -  df_t0.loc[df_t0["dataItemId"] == 3160]["data_item"].iloc[0] + 
            df_t0.loc[df_t0["dataItemId"] == 3148]["data_item"].iloc[0] - df_t0.loc[df_t0["dataItemId"] == 21678]["data_item"].iloc[0]) 
except:
    pension = 0 

# PPI t0 * invtentory t-1

try:
    ppi = -0.025
    fifo_profits = ppi*df_t1.loc[df_t1["dataItemId"] == 1043]["data_item"].iloc[0]
except:
    fifo_profits = 0

gross_cash_flow = (net_income + d_and_a - interest_expense + r_and_d_expense + rental_expense - minority_interest - 
                   associate_jv_income + special_items + monetary_holding + fifo_profits + pension)


print(net_income/1000000)
print(d_and_a/1000000)
print(interest_expense/1000000)
print(r_and_d_expense/1000000)
print(rental_expense/1000000)
print(minority_interest/1000000)
print(associate_jv_income/1000000)
print(special_items/1000000)
print(monetary_holding/1000000)
print(fifo_profits/1000000)
print(pension/1000000)

gross_cash_flow/1000000

784.678
173.557
-25.685
643.038
15.358
-1.949
0.0
14.687
0.0
-9.844125
0.0


1649.107875

In [538]:
#Depreciating Assets

#Asset Delayering

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_GPPE = 1169
# IQ_DA_SUPPL_CF = 2171


line_item = ["1169", "2171"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year, str(int(year) - 1), str(int(year) - 2), str(int(year) - 3), str(int(year) - 4), str(int(year) - 5)]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_inv_t0 = pd.read_sql(query_str, conn)

            
#years calc for delayering

year_0 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

year_1 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

year_2 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

delayering_years = int(round(median([year_0, year_1, year_2]),0))


#nominal growth rate calc

ppe_1 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0]) - 1

ppe_2 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/2) - 1

ppe_3 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-3)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/3) - 1

ppe_4 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-4)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/4) - 1

ppe_5 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-5)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/5) - 1

nominal_growth = (ppe_1 + ppe_2 + ppe_3 + ppe_4 + ppe_5) / 5

#Asset Layers (capex)

inflation = 0.025

gppe_inflation_adjustment = 0

#first year capex

gppe_0 = df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0]
capex_1st_year = gppe_0 / ((1+nominal_growth)**delayering_years-1) * nominal_growth

asset_layer = capex_1st_year

#calculate total inflation adjustment

for i in range (delayering_years-1, -1, -1):
    gppe_inflation_adjustment += asset_layer*(1+inflation)**i-asset_layer
    asset_layer = asset_layer*(1+nominal_growth)
        

        
#Capitalized leases

nomninal_bond_rate = 0.05

expected_inflation_rate = 0.025

asset_life = delayering_years

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_NET_RENTAL_EXP_FN = 3170

line_item = ["3170"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_cap_lease_t0 = pd.read_sql(query_str, conn)


#Present Value cals

real_debt_rate = nomninal_bond_rate - expected_inflation_rate
lease_expense = df_cap_lease_t0.loc[(df_cap_lease_t0["fiscalYear"] == int(year))&(df_cap_lease_t0["dataItemId"] == 3170)]["data_item"].iloc[0]

capitalized_lease = -np.pv(rate=real_debt_rate, nper=asset_life, pmt=lease_expense)


#Capitalized R&D over 5 years (need to change sql query if it changes from 5 years)

inflation = 0.025
inflation_factor = 1

rd_inflation_adj = 0
rd = 0

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_RD_EXP_FN = 3168

line_item = ["3168"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year, str(int(year) - 1), str(int(year) - 2), str(int(year) - 3), str(int(year) - 4)]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_rd = pd.read_sql(query_str, conn)

df_rd.loc[(df_rd["fiscalYear"] == int(year))&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]
for i in range(0,5):
    rd_inflation_adj += df_rd.loc[(df_rd["fiscalYear"] == int(year)-i)&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]*inflation_factor
    inflation_factor = inflation_factor*(1+inflation)
    rd += df_rd.loc[(df_rd["fiscalYear"] == int(year)-i)&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]
    

    
    
#total inflation adjusted depreciating assets

depr_assets = gppe_0 + gppe_inflation_adjustment + capitalized_lease + rd_inflation_adj


#non-depreciating assets

#net monetary assets

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_CASH_ST_INVEST = 1002
# IQ_TOTAL_RECEIV = 1001
# IQ_PREPAID_EXP = 1212
# IQ_TOTAL_CL = 1009
# IQ_ST_DEBT = 1046
# IQ_CURRENT_PORT = 1279

line_item = ["1002", "1001", "1212", "1009", "1046", "1279"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_monetary_assets = pd.read_sql(query_str, conn)

#Cash + receivables + prepaid expenses

current_assets_ex_inv = (df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1002)]["data_item"].iloc[0]
                        + df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1001)]["data_item"].iloc[0]
                        + df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1212)]["data_item"].iloc[0])

#Total current liabilites - short term debt - current portion of long-term debt

current_liab_ex_debt = (df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1009)]["data_item"].iloc[0]
                        - df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1046)]["data_item"].iloc[0]
                        - df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1279)]["data_item"].iloc[0])

#net monetary assets

net_monetary_assets = current_assets_ex_inv - current_liab_ex_debt


#inventory and other tangible assets

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_INVENTORY = 1043
# IQ_OTHER_ASSETS = 1272
# IQ_OTHER_CA = 1267

line_item = ["1043", "1272", "1267"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_monetary_assets = pd.read_sql(query_str, conn)

#inventory

inventory = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1043)]["data_item"].iloc[0]

#Other tangible assets

try:
    other_assets = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1272)]["data_item"].iloc[0]
except:
    other_assets = 0

try:
    other_current_assets = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1267)]["data_item"].iloc[0]
except:
    other_current_assets = 0
    
other_tangible_assets = other_assets + other_current_assets

#non-depreciating assets

non_depreciating_assets = net_monetary_assets + inventory + other_tangible_assets


#inflation adjusted gross investment

inflation_adj_gross_investment = depr_assets + non_depreciating_assets



#Project Life

#inflation adjusted gross plant

gppe_inflation_adj = gppe_0 + gppe_inflation_adjustment
gppe_life = delayering_years
implied_gppe_depr = gppe_inflation_adj / gppe_life

#capitalized operating leases

op_lease_life = delayering_years
implied_op_lease_depr = capitalized_lease / op_lease_life

#capitalized R&D at 5 years (needs to be consistent with years used in R&D calc)

implied_rd_depr = rd_inflation_adj / 5

#blended company asset life

blended_asset_life = round((gppe_inflation_adj + capitalized_lease + rd_inflation_adj) / (implied_gppe_depr + implied_op_lease_depr + implied_rd_depr),1)


#CFROI Calc

#Gross Investments (-PV) 

inflation_adj_gross_investment = depr_assets + non_depreciating_assets

#Non-Depreciating Assets (FV)

non_depreciating_assets

#Inflation Adjusted Gross Cash Flow (PMT)

gross_cash_flow

#Blended Asset Life (Period)

blended_asset_life

#CFROI

cfroi = np.rate(nper=int(blended_asset_life), pmt=int(gross_cash_flow), pv=-int(inflation_adj_gross_investment), 
                fv=int(non_depreciating_assets))
cfroi

15339.88


In [402]:
year = "2018"
ticker = "313055"

for i in b:

    #Gross Cash Flow calcs

    #Gross Cash Flow t0

    #company IDs
    ciq_id = [i]
    
    #list of line items

    # IQ_NI_CF = 2150
    # IQ_DA_CF = 2160
    # IQ_INTEREST_EXP = 82
    # IQ_RD_EXP_FN = 3168
    # IQ_NET_RENTAL_EXP_FN = 3170
    # IQ_PENSION_TOTAL_EXP = 3140
    # IQ_PENSION_SERVICE_COST = 3160
    # IQ_PENSION_INTEREST_COST = 3148
    # IQ_NONCASH_PENSION_EXP = 21678
    # IQ_MINORITY_INTEREST_IS = 83
    # IQ_INC_EQUITY = 47
    # IQ_TOTAL_UNUSUAL = 19

    line_item = ["2150", "2160", "82", "3168", "3170", "3140", "3160", "3148", "21678", "83", "47", "19"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = year

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     fiscal_year, financial_flag)


    df_t0 = pd.read_sql(query_str, conn)


    #Gross Cash Flow t-1

    #company IDs
    ciq_id = [i]

    #list of line items

    # IQ_CASH_ST_INVEST = 1002
    # IQ_TOTAL_RECEIV = 1001
    # IQ_PREPAID_EXP = 1212
    # IQ_TOTAL_CL = 1009
    # IQ_ST_DEBT = 1046
    # IQ_CURRENT_PORT = 1279
    # IQ_INVENTORY = 1043


    line_item = ["1002", "1001", "1212", "1009", "1046", "1043", "1279"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = str(int(year) - 1)

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     fiscal_year, financial_flag)


    df_t1 = pd.read_sql(query_str, conn)

    #potential to get better results by using try/except for all values

    try:
        net_income = df_t0.loc[df_t0["dataItemId"] == 2150]["data_item"].iloc[0]
    except: 
        net_income = 0 

    try:
        d_and_a = df_t0.loc[df_t0["dataItemId"] == 2160] ["data_item"].iloc[0]
    except:
        d_and_a = 0
    
    try:
        interest_expense =  df_t0.loc[df_t0["dataItemId"] == 82]["data_item"].iloc[0]
    except:
        interest_expense = 0 

    try:
        r_and_d_expense =  df_t0.loc[df_t0["dataItemId"] == 3168]["data_item"].iloc[0]
    except:
        r_and_d_expense = 0
    
    try:
        rental_expense =  df_t0.loc[df_t0["dataItemId"] == 3170]["data_item"].iloc[0]
    except:
        rental_expense = 0

    try:
        minority_interest = df_t0.loc[df_t0["dataItemId"] == 83]["data_item"].iloc[0]
    except:
        minority_interest = 0

    try:
        associate_jv_income = df_t0.loc[df_t0["dataItemId"] == 47]["data_item"].iloc[0]
    except: 
        associate_jv_income = 0

    try:
        special_items = df_t0.loc[df_t0["dataItemId"] == 19]["data_item"].iloc[0]
    except:
        special_items = 0

    # -gdp t0*(Cash & ST Invt + Receiv + prepaid exp - current liab + short term debt + current portion debt)t-1
    try:
        gdp = 0.022
        monetary_holding = -gdp*(df_t1.loc[df_t1["dataItemId"] == 1002]["data_item"].iloc[0] + df_t1.loc[df_t1["dataItemId"] == 1001]["data_item"].iloc[0]
                                + df_t1.loc[df_t1["dataItemId"] == 1212]["data_item"].iloc[0] -  df_t1.loc[df_t1["dataItemId"] == 1009]["data_item"].iloc[0] 
                                + df_t1.loc[df_t1["dataItemId"] == 1046]["data_item"].iloc[0] + df_t1.loc[df_t1["dataItemId"] == 1279]["data_item"].iloc[0])
    except:
        monetary_holding = 0

    #total pension cost - service cost + interest cost - noncash expense
    try:
        pension =  (df_t0.loc[df_t0["dataItemId"] == 3140]["data_item"].iloc[0] -  df_t0.loc[df_t0["dataItemId"] == 3160]["data_item"].iloc[0] + 
                df_t0.loc[df_t0["dataItemId"] == 3148]["data_item"].iloc[0] - df_t0.loc[df_t0["dataItemId"] == 21678]["data_item"].iloc[0]) 
    except:
        pension = 0 

    # PPI t0 * invtentory t-1
    
    try:
        ppi = -0.025
        fifo_profits = ppi*df_t1.loc[df_t1["dataItemId"] == 1043]["data_item"].iloc[0]
    except:
        fifo_profits = 0

    gross_cash_flow = (net_income + d_and_a + interest_expense + r_and_d_expense + rental_expense + minority_interest + 
                       associate_jv_income + special_items + monetary_holding + fifo_profits + pension)

    print(gross_cash_flow/1000000)

42386.475
82528.625
55847.825
63810.275
210060.094104
10295.85
37320.0
85900.545
144415.953
31670.683


In [478]:
#Depreciating Assets

#Asset Delayering

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_GPPE = 1169
# IQ_DA_SUPPL_CF = 2171


line_item = ["1169", "2171"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year, str(int(year) - 1), str(int(year) - 2), str(int(year) - 3), str(int(year) - 4), str(int(year) - 5)]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_inv_t0 = pd.read_sql(query_str, conn)

            
#years calc for delayering

year_0 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

year_1 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

year_2 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

delayering_years = int(round(median([year_0, year_1, year_2]),0))


#nominal growth rate calc

ppe_1 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0]) - 1

ppe_2 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/2) - 1

ppe_3 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-3)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/3) - 1

ppe_4 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-4)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/4) - 1

ppe_5 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-5)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/5) - 1

nominal_growth = (ppe_1 + ppe_2 + ppe_3 + ppe_4 + ppe_5) / 5

nominal_growth

0.21340895760674225

In [562]:
#Asset Layers (capex)

inflation = 0.025

gppe_inflation_adjustment = 0

#first year capex

gppe_0 = df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0]
capex_1st_year = gppe_0 / ((1+nominal_growth)**delayering_years-1) * nominal_growth

asset_layer = capex_1st_year

#calculate total inflation adjustment

for i in range (delayering_years-1, -1, -1):
    gppe_inflation_adjustment += asset_layer*(1+inflation)**i-asset_layer
    asset_layer = asset_layer*(1+nominal_growth)
        
gppe_inflation_adjustment

185481568.47579646

In [511]:
#Capitalized leases

nomninal_bond_rate = 0.05

expected_inflation_rate = 0.025

asset_life = delayering_years

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_NET_RENTAL_EXP_FN = 3170

line_item = ["3170"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_cap_lease_t0 = pd.read_sql(query_str, conn)


#Present Value cals

real_debt_rate = nomninal_bond_rate - expected_inflation_rate
lease_expense = df_cap_lease_t0.loc[(df_cap_lease_t0["fiscalYear"] == int(year))&(df_cap_lease_t0["dataItemId"] == 3170)]["data_item"].iloc[0]

capitalized_lease = -np.pv(rate=real_debt_rate, nper=asset_life, pmt=lease_expense)
capitalized_lease

163402743.34952626

In [581]:
#Capitalized R&D over 5 years (need to change sql query if it changes from 5 years)

inflation = 0.025
inflation_factor = 1

#total inflation adjusted r&d
rd_inflation_adj = 0

#used to check total r&d that is not inflation adjusted
rd = 0

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_RD_EXP_FN = 3168

line_item = ["3168"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year, str(int(year) - 1), str(int(year) - 2), str(int(year) - 3), str(int(year) - 4)]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_rd = pd.read_sql(query_str, conn)

for i in range(0,5):
    rd_inflation_adj += df_rd.loc[(df_rd["fiscalYear"] == int(year)-i)&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]*inflation_factor
    inflation_factor = inflation_factor*(1+inflation)
    rd += df_rd.loc[(df_rd["fiscalYear"] == int(year)-i)&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]
    
print(rd_inflation_adj/1000000)

1701.690098515625
1627.5


In [564]:
#total inflation adjusted depreciating assets

depr_assets = gppe_0 + gppe_inflation_adjustment + capitalized_lease + rd_inflation_adj


4600274410.340948

In [546]:
#non-depreciating assets

#net monetary assets

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_CASH_ST_INVEST = 1002
# IQ_TOTAL_RECEIV = 1001
# IQ_PREPAID_EXP = 1212
# IQ_TOTAL_CL = 1009
# IQ_ST_DEBT = 1046
# IQ_CURRENT_PORT = 1279

line_item = ["1002", "1001", "1212", "1009", "1046", "1279"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_monetary_assets = pd.read_sql(query_str, conn)

#Cash + receivables + prepaid expenses

current_assets_ex_inv = (df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1002)]["data_item"].iloc[0]
                        + df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1001)]["data_item"].iloc[0]
                        + df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1212)]["data_item"].iloc[0])

#Total current liabilites - short term debt - current portion of long-term debt

current_liab_ex_debt = (df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1009)]["data_item"].iloc[0]
                        - df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1046)]["data_item"].iloc[0]
                        - df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1279)]["data_item"].iloc[0])

#net monetary assets

net_monetary_assets = current_assets_ex_inv - current_liab_ex_debt

In [544]:
current_assets_ex_inv

15881000000.0

In [545]:
current_liab_ex_debt

68859000000.0

In [547]:
net_monetary_assets

-52978000000.0

In [551]:
#non-depreciating assets

#inventory and other tangible assets

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_INVENTORY = 1043
# IQ_OTHER_ASSETS = 1272
# IQ_OTHER_CA = 1267

line_item = ["1043", "1272", "1267"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_monetary_assets = pd.read_sql(query_str, conn)

#inventory

inventory = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1043)]["data_item"].iloc[0]

#Other tangible assets

try:
    other_assets = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1272)]["data_item"].iloc[0]
except:
    other_assets = 0

try:
    other_current_assets = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1267)]["data_item"].iloc[0]
except:
    other_current_assets = 0
    
other_tangible_assets = other_assets + other_current_assets

#non-depreciating assets

non_depreciating_assets = net_monetary_assets + inventory + other_tangible_assets

In [558]:
#inflation adjusted gross investment

inflation_adj_gross_investment = depr_assets + non_depreciating_assets

In [570]:
#Project Life

#inflation adjusted gross plant

gppe_inflation_adj = gppe_0 + gppe_inflation_adjustment
gppe_life = delayering_years
implied_gppe_depr = gppe_inflation_adj / gppe_life

#capitalized operating leases

op_lease_life = delayering_years
implied_op_lease_depr = capitalized_lease / op_lease_life

#capitalized R&D at 5 years (needs to be consistent with years used in R&D calc)

implied_rd_depr = rd_inflation_adj / 5

#blended company asset life

blended_asset_life = round((gppe_inflation_adj + capitalized_lease + rd_inflation_adj) / (implied_gppe_depr + implied_op_lease_depr + implied_rd_depr),1)

6.9

In [574]:
#CFROI Calc

#Gross Investments (-PV) 

inflation_adj_gross_investment = depr_assets + non_depreciating_assets

#Non-Depreciating Assets (FV)

non_depreciating_assets

#Inflation Adjusted Gross Cash Flow (PMT)

gross_cash_flow

#Blended Asset Life (Period)

blended_asset_life

#CFROI

cfroi = np.rate(nper=int(blended_asset_life), pmt=int(gross_cash_flow), pv=-int(inflation_adj_gross_investment), 
                fv=int(non_depreciating_assets))
cfroi

-1.930699816408999

In [673]:
year = test_year
ticker = test_ticker


#Gross Cash Flow calcs

#Gross Cash Flow t0

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_NI_CF = 2150
# IQ_DA_CF = 2160
# IQ_INTEREST_EXP = 82
# IQ_RD_EXP_FN = 3168
# IQ_NET_RENTAL_EXP_FN = 3170
# IQ_PENSION_TOTAL_EXP = 3140
# IQ_PENSION_SERVICE_COST = 3160
# IQ_PENSION_INTEREST_COST = 3148
# IQ_NONCASH_PENSION_EXP = 21678
# IQ_MINORITY_INTEREST_IS = 83
# IQ_INC_EQUITY = 47
# IQ_TOTAL_UNUSUAL = 19

line_item = ["2150", "2160", "82", "3168", "3170", "3140", "3160", "3148", "21678", "83", "47", "19"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = year

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)


df_t0 = pd.read_sql(query_str, conn)


#Gross Cash Flow t-1

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_CASH_ST_INVEST = 1002
# IQ_TOTAL_RECEIV = 1001
# IQ_PREPAID_EXP = 1212
# IQ_TOTAL_CL = 1009
# IQ_ST_DEBT = 1046
# IQ_CURRENT_PORT = 1279
# IQ_INVENTORY = 1043


line_item = ["1002", "1001", "1212", "1009", "1046", "1043", "1279"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = str(int(year) - 1)

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 fiscal_year, financial_flag)


df_t1 = pd.read_sql(query_str, conn)

#potential to get better results by using try/except for all values

try:
    net_income = df_t0.loc[df_t0["dataItemId"] == 2150]["data_item"].iloc[0]
except: 
    net_income = 0 

try:
    d_and_a = df_t0.loc[df_t0["dataItemId"] == 2160] ["data_item"].iloc[0]
except:
    d_and_a = 0

try:
    interest_expense =  df_t0.loc[df_t0["dataItemId"] == 82]["data_item"].iloc[0]
except:
    interest_expense = 0 

try:
    r_and_d_expense =  df_t0.loc[df_t0["dataItemId"] == 3168]["data_item"].iloc[0]
except:
    r_and_d_expense = 0

try:
    rental_expense =  df_t0.loc[df_t0["dataItemId"] == 3170]["data_item"].iloc[0]
except:
    rental_expense = 0

try:
    minority_interest = df_t0.loc[df_t0["dataItemId"] == 83]["data_item"].iloc[0]
except:
    minority_interest = 0

try:
    associate_jv_income = df_t0.loc[df_t0["dataItemId"] == 47]["data_item"].iloc[0]
except: 
    associate_jv_income = 0

try:
    special_items = df_t0.loc[df_t0["dataItemId"] == 19]["data_item"].iloc[0]
except:
    special_items = 0

# -gdp t0*(Cash & ST Invt + Receiv + prepaid exp - current liab + short term debt + current portion debt)t-1
try:
    gdp = 0.022
    monetary_holding = -gdp*(df_t1.loc[df_t1["dataItemId"] == 1002]["data_item"].iloc[0] + df_t1.loc[df_t1["dataItemId"] == 1001]["data_item"].iloc[0]
                            + df_t1.loc[df_t1["dataItemId"] == 1212]["data_item"].iloc[0] -  df_t1.loc[df_t1["dataItemId"] == 1009]["data_item"].iloc[0] 
                            + df_t1.loc[df_t1["dataItemId"] == 1046]["data_item"].iloc[0] + df_t1.loc[df_t1["dataItemId"] == 1279]["data_item"].iloc[0])
except:
    monetary_holding = 0

#total pension cost - service cost + interest cost - noncash expense
try:
    pension =  (df_t0.loc[df_t0["dataItemId"] == 3140]["data_item"].iloc[0] -  df_t0.loc[df_t0["dataItemId"] == 3160]["data_item"].iloc[0] + 
            df_t0.loc[df_t0["dataItemId"] == 3148]["data_item"].iloc[0] - df_t0.loc[df_t0["dataItemId"] == 21678]["data_item"].iloc[0]) 
except:
    pension = 0 

# PPI t0 * invtentory t-1

try:
    ppi = -0.025
    fifo_profits = ppi*df_t1.loc[df_t1["dataItemId"] == 1043]["data_item"].iloc[0]
except:
    fifo_profits = 0

gross_cash_flow = (net_income + d_and_a - interest_expense + r_and_d_expense + rental_expense - minority_interest - 
                   associate_jv_income + special_items + monetary_holding + fifo_profits + pension)







#Depreciating Assets

#Asset Delayering

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_GPPE = 1169
# IQ_DA_SUPPL_CF = 2171


line_item = ["1169", "2171"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year, str(int(year) - 1), str(int(year) - 2), str(int(year) - 3), str(int(year) - 4), str(int(year) - 5)]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_inv_t0 = pd.read_sql(query_str, conn)

            
#years calc for delayering

year_0 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

year_1 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

year_2 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

delayering_years = int(round(median([year_0, year_1, year_2]),0))


#nominal growth rate calc

ppe_1 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0]) - 1

ppe_2 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/2) - 1

ppe_3 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-3)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/3) - 1

ppe_4 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-4)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/4) - 1

ppe_5 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
          / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-5)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/5) - 1

nominal_growth = (ppe_1 + ppe_2 + ppe_3 + ppe_4 + ppe_5) / 5

#Asset Layers (capex)

inflation = 0.025

gppe_inflation_adjustment = 0

#first year capex

gppe_0 = df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0]
capex_1st_year = gppe_0 / ((1+nominal_growth)**delayering_years-1) * nominal_growth

asset_layer = capex_1st_year

#calculate total inflation adjustment

for i in range (delayering_years-1, -1, -1):
    gppe_inflation_adjustment += asset_layer*(1+inflation)**i-asset_layer
    asset_layer = asset_layer*(1+nominal_growth)
        

        
#Capitalized leases

nomninal_bond_rate = 0.05

expected_inflation_rate = 0.025

asset_life = delayering_years

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_NET_RENTAL_EXP_FN = 3170

line_item = ["3170"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_cap_lease_t0 = pd.read_sql(query_str, conn)


#Present Value cals

real_debt_rate = nomninal_bond_rate - expected_inflation_rate
lease_expense = df_cap_lease_t0.loc[(df_cap_lease_t0["fiscalYear"] == int(year))&(df_cap_lease_t0["dataItemId"] == 3170)]["data_item"].iloc[0]

capitalized_lease = -np.pv(rate=real_debt_rate, nper=asset_life, pmt=lease_expense)


#Capitalized R&D over 5 years (need to change sql query if it changes from 5 years)

inflation = 0.025
inflation_factor = 1

#total inflation adjusted r&d
rd_inflation_adj = 0

#used to check total r&d that is not inflation adjusted
rd = 0

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_RD_EXP_FN = 3168

line_item = ["3168"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year, str(int(year) - 1), str(int(year) - 2), str(int(year) - 3), str(int(year) - 4)]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_rd = pd.read_sql(query_str, conn)

for i in range(0,5):
    
    try:
        rd_inflation_adj += df_rd.loc[(df_rd["fiscalYear"] == int(year)-i)&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]*inflation_factor
    except:
        rd_inflation_adj += 0 
        
    inflation_factor = inflation_factor*(1+inflation)
    
    try:
        rd += df_rd.loc[(df_rd["fiscalYear"] == int(year)-i)&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]
    except:
        rd += 0
    

    
    
#total inflation adjusted depreciating assets

depr_assets = gppe_0 + gppe_inflation_adjustment + capitalized_lease + rd_inflation_adj


#non-depreciating assets

#net monetary assets

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_CASH_ST_INVEST = 1002
# IQ_TOTAL_RECEIV = 1001
# IQ_PREPAID_EXP = 1212
# IQ_TOTAL_CL = 1009
# IQ_ST_DEBT = 1046
# IQ_CURRENT_PORT = 1279

line_item = ["1002", "1001", "1212", "1009", "1046", "1279"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_monetary_assets = pd.read_sql(query_str, conn)

#Cash + receivables + prepaid expenses

try:
    cash = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1002)]["data_item"].iloc[0]
except:
    cash = 0

try:
    receiv = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1001)]["data_item"].iloc[0]
except:
    receiv = 0
    
try:
    prepaid = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1212)]["data_item"].iloc[0]
except:
    prepaid = 0
    
current_assets_ex_inv = cash+receiv+prepaid

#Total current liabilites - short term debt - current portion of long-term debt


try:
    current_liab = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1009)]["data_item"].iloc[0]
except:
    current_liab = 0
    
try:
    short_term = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1046)]["data_item"].iloc[0]
except:
    short_term = 0
    
try:
    current_portion = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1279)]["data_item"].iloc[0]
except:
    current_portion = 0 
    
current_liab_ex_debt = current_liab - short_term - current_portion

#net monetary assets

net_monetary_assets = current_assets_ex_inv - current_liab_ex_debt


#inventory and other tangible assets

#company IDs
ciq_id = [ticker]

#list of line items

# IQ_INVENTORY = 1043
# IQ_OTHER_ASSETS = 1272
# IQ_OTHER_CA = 1267

line_item = ["1043", "1272", "1267"]


#annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
period_type = '1'

#fiscal year
fiscal_year = [year]

#latest filing
financial_flag = '1'



query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
            ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
            'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
            'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
            'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
            'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
            'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
            'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
            'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
            'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                 ', '.join(fiscal_year), financial_flag)

df_other_assets = pd.read_sql(query_str, conn)

#inventory

inventory = df_other_assets.loc[(df_other_assets["fiscalYear"] == int(year))&(df_other_assets["dataItemId"] == 1043)]["data_item"].iloc[0]

#Other tangible assets

try:
    other_assets = df_other_assets.loc[(df_other_assets["fiscalYear"] == int(year))&(df_other_assets["dataItemId"] == 1272)]["data_item"].iloc[0]
except:
    other_assets = 0

try:
    other_current_assets = df_other_assets.loc[(df_other_assets["fiscalYear"] == int(year))&(df_other_assets["dataItemId"] == 1267)]["data_item"].iloc[0]
except:
    other_current_assets = 0
    
other_tangible_assets = other_assets + other_current_assets

#non-depreciating assets

non_depreciating_assets = net_monetary_assets + inventory + other_tangible_assets


#inflation adjusted gross investment

inflation_adj_gross_investment = depr_assets + non_depreciating_assets



#Project Life

#inflation adjusted gross plant

gppe_inflation_adj = gppe_0 + gppe_inflation_adjustment
gppe_life = delayering_years
implied_gppe_depr = gppe_inflation_adj / gppe_life

#capitalized operating leases

op_lease_life = delayering_years
implied_op_lease_depr = capitalized_lease / op_lease_life

#capitalized R&D at 5 years (needs to be consistent with years used in R&D calc)

implied_rd_depr = rd_inflation_adj / 5

#blended company asset life

blended_asset_life = round((gppe_inflation_adj + capitalized_lease + rd_inflation_adj) / (implied_gppe_depr + implied_op_lease_depr + implied_rd_depr),1)


#CFROI Calc

#Gross Investments (-PV) 

inflation_adj_gross_investment = depr_assets + non_depreciating_assets

#Non-Depreciating Assets (FV)

non_depreciating_assets

#Inflation Adjusted Gross Cash Flow (PMT)

gross_cash_flow

#Blended Asset Life (Period)

blended_asset_life

#CFROI

cfroi = np.rate(nper=int(blended_asset_life), pmt=int(gross_cash_flow), pv=-int(inflation_adj_gross_investment), 
                fv=int(non_depreciating_assets))

print("Year: {}".format(year))
print("CFROI: {:.2%}".format(cfroi))
print("Gross Investment: {:.5} billion".format(inflation_adj_gross_investment/1000000000))
print("Non-Depreciating Assets: {:.5} billion".format(non_depreciating_assets/1000000000))
print("Gross Cash Flow: {:.5} billion".format(gross_cash_flow/1000000000))
print("Asset Life: {:.3} years".format(blended_asset_life))


Year: 2018
CFROI: 49.25%
Gross Investment: 2.1246 billion
Non-Depreciating Assets: 1.476 billion
Gross Cash Flow: 1.0552 billion
Asset Life: 9.0 years


In [672]:
test_year = "2018"
test_ticker = "1462309"

In [688]:
for i in ["2013","2014", "2015", "2016", "2017", "2018"]:
    
    year = i
    ticker = test_ticker


    #Gross Cash Flow calcs

    #Gross Cash Flow t0

    #company IDs
    ciq_id = [ticker]

    #list of line items

    # IQ_NI_CF = 2150
    # IQ_DA_CF = 2160
    # IQ_INTEREST_EXP = 82
    # IQ_RD_EXP_FN = 3168
    # IQ_NET_RENTAL_EXP_FN = 3170
    # IQ_PENSION_TOTAL_EXP = 3140
    # IQ_PENSION_SERVICE_COST = 3160
    # IQ_PENSION_INTEREST_COST = 3148
    # IQ_NONCASH_PENSION_EXP = 21678
    # IQ_MINORITY_INTEREST_IS = 83
    # IQ_INC_EQUITY = 47
    # IQ_TOTAL_UNUSUAL = 19

    line_item = ["2150", "2160", "82", "3168", "3170", "3140", "3160", "3148", "21678", "83", "47", "19"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = year

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     fiscal_year, financial_flag)


    df_t0 = pd.read_sql(query_str, conn)


    #Gross Cash Flow t-1

    #company IDs
    ciq_id = [ticker]

    #list of line items

    # IQ_CASH_ST_INVEST = 1002
    # IQ_TOTAL_RECEIV = 1001
    # IQ_PREPAID_EXP = 1212
    # IQ_TOTAL_CL = 1009
    # IQ_ST_DEBT = 1046
    # IQ_CURRENT_PORT = 1279
    # IQ_INVENTORY = 1043


    line_item = ["1002", "1001", "1212", "1009", "1046", "1043", "1279"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = str(int(year) - 1)

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear={3} AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     fiscal_year, financial_flag)


    df_t1 = pd.read_sql(query_str, conn)

    #potential to get better results by using try/except for all values

    try:
        net_income = df_t0.loc[df_t0["dataItemId"] == 2150]["data_item"].iloc[0]
    except: 
        net_income = 0 

    try:
        d_and_a = df_t0.loc[df_t0["dataItemId"] == 2160] ["data_item"].iloc[0]
    except:
        d_and_a = 0

    try:
        interest_expense =  df_t0.loc[df_t0["dataItemId"] == 82]["data_item"].iloc[0]
    except:
        interest_expense = 0 

    try:
        r_and_d_expense =  df_t0.loc[df_t0["dataItemId"] == 3168]["data_item"].iloc[0]
    except:
        r_and_d_expense = 0

    try:
        rental_expense =  df_t0.loc[df_t0["dataItemId"] == 3170]["data_item"].iloc[0]
    except:
        rental_expense = 0

    try:
        minority_interest = df_t0.loc[df_t0["dataItemId"] == 83]["data_item"].iloc[0]
    except:
        minority_interest = 0

    try:
        associate_jv_income = df_t0.loc[df_t0["dataItemId"] == 47]["data_item"].iloc[0]
    except: 
        associate_jv_income = 0

    try:
        special_items = df_t0.loc[df_t0["dataItemId"] == 19]["data_item"].iloc[0]
    except:
        special_items = 0

    # -gdp t0*(Cash & ST Invt + Receiv + prepaid exp - current liab + short term debt + current portion debt)t-1
    try:
        gdp = 0.022
        monetary_holding = -gdp*(df_t1.loc[df_t1["dataItemId"] == 1002]["data_item"].iloc[0] + df_t1.loc[df_t1["dataItemId"] == 1001]["data_item"].iloc[0]
                                + df_t1.loc[df_t1["dataItemId"] == 1212]["data_item"].iloc[0] -  df_t1.loc[df_t1["dataItemId"] == 1009]["data_item"].iloc[0] 
                                + df_t1.loc[df_t1["dataItemId"] == 1046]["data_item"].iloc[0] + df_t1.loc[df_t1["dataItemId"] == 1279]["data_item"].iloc[0])
    except:
        monetary_holding = 0

    #total pension cost - service cost + interest cost - noncash expense
    try:
        pension =  (df_t0.loc[df_t0["dataItemId"] == 3140]["data_item"].iloc[0] -  df_t0.loc[df_t0["dataItemId"] == 3160]["data_item"].iloc[0] + 
                df_t0.loc[df_t0["dataItemId"] == 3148]["data_item"].iloc[0] - df_t0.loc[df_t0["dataItemId"] == 21678]["data_item"].iloc[0]) 
    except:
        pension = 0 

    # PPI t0 * invtentory t-1

    try:
        ppi = -0.025
        fifo_profits = ppi*df_t1.loc[df_t1["dataItemId"] == 1043]["data_item"].iloc[0]
    except:
        fifo_profits = 0

    gross_cash_flow = (net_income + d_and_a - interest_expense + r_and_d_expense + rental_expense - minority_interest - 
                       associate_jv_income + special_items + monetary_holding + fifo_profits + pension)







    #Depreciating Assets

    #Asset Delayering

    #company IDs
    ciq_id = [ticker]

    #list of line items

    # IQ_GPPE = 1169
    # IQ_DA_SUPPL_CF = 2171


    line_item = ["1169", "2171"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = [year, str(int(year) - 1), str(int(year) - 2), str(int(year) - 3), str(int(year) - 4), str(int(year) - 5)]

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     ', '.join(fiscal_year), financial_flag)

    df_inv_t0 = pd.read_sql(query_str, conn)


    #years calc for delayering

    year_0 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
              / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

    year_1 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
              / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

    year_2 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
              / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 2171)]["data_item"].iloc[0])

    delayering_years = int(round(median([year_0, year_1, year_2]),0))


    #nominal growth rate calc

    ppe_1 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
              / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-1)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0]) - 1

    ppe_2 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
              / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-2)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/2) - 1

    ppe_3 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
              / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-3)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/3) - 1

    ppe_4 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
              / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-4)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/4) - 1

    ppe_5 = (df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0] 
              / df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year)-5)&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0])**(1/5) - 1

    nominal_growth = (ppe_1 + ppe_2 + ppe_3 + ppe_4 + ppe_5) / 5

    #Asset Layers (capex)

    inflation = 0.025

    gppe_inflation_adjustment = 0

    #first year capex

    gppe_0 = df_inv_t0.loc[(df_inv_t0["fiscalYear"] == int(year))&(df_inv_t0["dataItemId"] == 1169)]["data_item"].iloc[0]
    capex_1st_year = gppe_0 / ((1+nominal_growth)**delayering_years-1) * nominal_growth

    asset_layer = capex_1st_year

    #calculate total inflation adjustment

    for i in range (delayering_years-1, -1, -1):
        gppe_inflation_adjustment += asset_layer*(1+inflation)**i-asset_layer
        asset_layer = asset_layer*(1+nominal_growth)



    #Capitalized leases

    nomninal_bond_rate = 0.05

    expected_inflation_rate = 0.025

    asset_life = delayering_years

    #company IDs
    ciq_id = [ticker]

    #list of line items

    # IQ_NET_RENTAL_EXP_FN = 3170

    line_item = ["3170"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = [year]

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     ', '.join(fiscal_year), financial_flag)

    df_cap_lease_t0 = pd.read_sql(query_str, conn)


    #Present Value cals

    real_debt_rate = nomninal_bond_rate - expected_inflation_rate
    lease_expense = df_cap_lease_t0.loc[(df_cap_lease_t0["fiscalYear"] == int(year))&(df_cap_lease_t0["dataItemId"] == 3170)]["data_item"].iloc[0]

    capitalized_lease = -np.pv(rate=real_debt_rate, nper=asset_life, pmt=lease_expense)


    #Capitalized R&D over 5 years (need to change sql query if it changes from 5 years)

    inflation = 0.025
    inflation_factor = 1

    #total inflation adjusted r&d
    rd_inflation_adj = 0

    #used to check total r&d that is not inflation adjusted
    rd = 0

    #company IDs
    ciq_id = [ticker]

    #list of line items

    # IQ_RD_EXP_FN = 3168

    line_item = ["3168"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = [year, str(int(year) - 1), str(int(year) - 2), str(int(year) - 3), str(int(year) - 4)]

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     ', '.join(fiscal_year), financial_flag)

    df_rd = pd.read_sql(query_str, conn)

    for i in range(0,5):

        try:
            rd_inflation_adj += df_rd.loc[(df_rd["fiscalYear"] == int(year)-i)&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]*inflation_factor
        except:
            rd_inflation_adj += 0 

        inflation_factor = inflation_factor*(1+inflation)

        try:
            rd += df_rd.loc[(df_rd["fiscalYear"] == int(year)-i)&(df_rd["dataItemId"] == 3168)]["data_item"].iloc[0]
        except:
            rd += 0




    #total inflation adjusted depreciating assets

    depr_assets = gppe_0 + gppe_inflation_adjustment + capitalized_lease + rd_inflation_adj


    #non-depreciating assets

    #net monetary assets

    #company IDs
    ciq_id = [ticker]

    #list of line items

    # IQ_CASH_ST_INVEST = 1002
    # IQ_TOTAL_RECEIV = 1001
    # IQ_PREPAID_EXP = 1212
    # IQ_TOTAL_CL = 1009
    # IQ_ST_DEBT = 1046
    # IQ_CURRENT_PORT = 1279

    line_item = ["1002", "1001", "1212", "1009", "1046", "1279"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = [year]

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     ', '.join(fiscal_year), financial_flag)

    df_monetary_assets = pd.read_sql(query_str, conn)

    #Cash + receivables + prepaid expenses

    try:
        cash = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1002)]["data_item"].iloc[0]
    except:
        cash = 0

    try:
        receiv = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1001)]["data_item"].iloc[0]
    except:
        receiv = 0

    try:
        prepaid = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1212)]["data_item"].iloc[0]
    except:
        prepaid = 0

    current_assets_ex_inv = cash+receiv+prepaid

    #Total current liabilites - short term debt - current portion of long-term debt


    try:
        current_liab = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1009)]["data_item"].iloc[0]
    except:
        current_liab = 0

    try:
        short_term = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1046)]["data_item"].iloc[0]
    except:
        short_term = 0

    try:
        current_portion = df_monetary_assets.loc[(df_monetary_assets["fiscalYear"] == int(year))&(df_monetary_assets["dataItemId"] == 1279)]["data_item"].iloc[0]
    except:
        current_portion = 0 

    current_liab_ex_debt = current_liab - short_term - current_portion

    #net monetary assets

    net_monetary_assets = current_assets_ex_inv - current_liab_ex_debt


    #inventory and other tangible assets

    #company IDs
    ciq_id = [ticker]

    #list of line items

    # IQ_INVENTORY = 1043
    # IQ_OTHER_ASSETS = 1272
    # IQ_OTHER_CA = 1267

    line_item = ["1043", "1272", "1267"]


    #annual data (1-annual; 2-Quarterly; 3-YTD; 4-LTM; 10-Semi-Annual; 17:Interim )
    period_type = '1'

    #fiscal year
    fiscal_year = [year]

    #latest filing
    financial_flag = '1'



    query_str = 'SELECT c.companyId, fi.periodEndDate, fi.filingDate, ' \
                ' fp.fiscalQuarter, fp.fiscalYear, fd.dataItemId, fd.dataItemValue * fu.unitTypeValue as data_item, fz.dataItemName ' \
                'FROM Xpressfeed..ciqCompany c JOIN Xpressfeed..ciqFinPeriod fp ON fp.companyId = c.companyId ' \
                'JOIN Xpressfeed..ciqFinInstance fi ON fi.financialPeriodId = fp.financialPeriodId ' \
                'JOIN Xpressfeed..ciqFinInstanceToCollection ic ON ic.financialInstanceId = fi.financialInstanceId ' \
                'JOIN Xpressfeed..ciqFinCollectionData fd ON fd.financialCollectionId = ic.financialCollectionId ' \
                'JOIN Xpressfeed..ciqDataItem fz ON fz.dataItemId = fd.dataItemId ' \
                'JOIN Xpressfeed..ciqFinUnitType fu ON fd.unitTypeId = fu.unitTypeId ' \
                'WHERE c.companyId IN ({0}) AND fd.dataItemId IN ({1}) AND fp.periodTypeId={2} AND ' \
                'fp.fiscalYear IN ({3}) AND fi.latestForFinancialPeriodFlag = {4}'.format(', '.join(ciq_id), ', '.join(line_item), period_type,
                                                                                     ', '.join(fiscal_year), financial_flag)

    df_other_assets = pd.read_sql(query_str, conn)

    #inventory

    inventory = df_other_assets.loc[(df_other_assets["fiscalYear"] == int(year))&(df_other_assets["dataItemId"] == 1043)]["data_item"].iloc[0]

    #Other tangible assets

    try:
        other_assets = df_other_assets.loc[(df_other_assets["fiscalYear"] == int(year))&(df_other_assets["dataItemId"] == 1272)]["data_item"].iloc[0]
    except:
        other_assets = 0

    try:
        other_current_assets = df_other_assets.loc[(df_other_assets["fiscalYear"] == int(year))&(df_other_assets["dataItemId"] == 1267)]["data_item"].iloc[0]
    except:
        other_current_assets = 0

    other_tangible_assets = other_assets + other_current_assets

    #non-depreciating assets

    non_depreciating_assets = net_monetary_assets + inventory + other_tangible_assets


    #inflation adjusted gross investment

    inflation_adj_gross_investment = depr_assets + non_depreciating_assets



    #Project Life

    #inflation adjusted gross plant

    gppe_inflation_adj = gppe_0 + gppe_inflation_adjustment
    gppe_life = delayering_years
    implied_gppe_depr = gppe_inflation_adj / gppe_life

    #capitalized operating leases

    op_lease_life = delayering_years
    implied_op_lease_depr = capitalized_lease / op_lease_life

    #capitalized R&D at 5 years (needs to be consistent with years used in R&D calc)

    implied_rd_depr = rd_inflation_adj / 5

    #blended company asset life

    blended_asset_life = round((gppe_inflation_adj + capitalized_lease + rd_inflation_adj) / (implied_gppe_depr + implied_op_lease_depr + implied_rd_depr),1)


    #CFROI Calc

    #Gross Investments (-PV) 

    inflation_adj_gross_investment = depr_assets + non_depreciating_assets

    #Non-Depreciating Assets (FV)

    non_depreciating_assets

    #Inflation Adjusted Gross Cash Flow (PMT)

    gross_cash_flow

    #Blended Asset Life (Period)

    blended_asset_life

    #CFROI

    cfroi = np.rate(nper=int(blended_asset_life), pmt=int(gross_cash_flow), pv=-int(inflation_adj_gross_investment), 
                    fv=int(non_depreciating_assets))

    print("Year: {}".format(year))
    print("CFROI: {:.2%}".format(cfroi))
    print("Gross Investment: {:.5} billion".format(inflation_adj_gross_investment/1000000000))
    print("Non-Depreciating Assets: {:.5} billion".format(non_depreciating_assets/1000000000))
    print("Gross Cash Flow: {:.5} billion".format(gross_cash_flow/1000000000))
    print("Asset Life: {:.3} years".format(blended_asset_life))


Year: 2013
CFROI: 8.28%
Gross Investment: 34.183 billion
Non-Depreciating Assets: 1.599 billion
Gross Cash Flow: 3.455 billion
Asset Life: 21.9 years
Year: 2014
CFROI: 7.72%
Gross Investment: 35.074 billion
Non-Depreciating Assets: 1.432 billion
Gross Cash Flow: 3.3985 billion
Asset Life: 21.9 years
Year: 2015
CFROI: 6.59%
Gross Investment: 33.729 billion
Non-Depreciating Assets: 1.447 billion
Gross Cash Flow: 3.045 billion
Asset Life: 20.0 years
Year: 2016
CFROI: 6.24%
Gross Investment: 34.757 billion
Non-Depreciating Assets: 1.544 billion
Gross Cash Flow: 3.0488 billion
Asset Life: 20.0 years
Year: 2017
CFROI: 5.24%
Gross Investment: 37.244 billion
Non-Depreciating Assets: 1.764 billion
Gross Cash Flow: 2.9184 billion
Asset Life: 21.9 years
Year: 2018
CFROI: 15.67%
Gross Investment: 58.285 billion
Non-Depreciating Assets: 8.71 billion
Gross Cash Flow: 9.4646 billion
Asset Life: 22.2 years


In [687]:
test_year = "2018"
test_ticker = "318049"

In [ ]:
#add in funcationality to printy company name and to enter ticker instead of the ciq code